<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 348.5/348.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 36.9/36.9 MB 7.7 MB/s eta 0:00:00
     -------------------------------------- 209.8/209.8 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 81.0/81.0 kB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 50.0/50.0 kB ? eta 0:00:00
     -------------------------------------- 147.1/147.1 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 95.2/95.2 kB 5.7 MB/s eta 0:00:00
     -------------------------------------- 112.6/112.6 kB 6.4 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11123 sha256=a4765e9ef9651121

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [19]:
#Импорты модулей
import pandas as pd

import lightgbm as lgb

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import optuna

import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import LinearSVC

SEED = 7777

[nltk_data] Downloading package wordnet to C:\Users\Вадим и
[nltk_data]     Света\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Вадим и Света\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv(r'C:\Users\Вадим и Света\Downloads\toxic_comments.csv')

data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [4]:
wnl = WordNetLemmatizer()

#Функция возврата тега слова
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

corpus = data['text'].values.astype('U')
corpus = pd.Series([re.sub(r'[^a-zA-Z0-9]', ' ', corpus[i]) for i in range(data.shape[0])]) #очистка текста

sentences=[]

#Цикл лемматизации слов и удаления пробелов
#for sentence in corpus: 
#    sentence_list = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
#    sentences.append(sentence_list)
    
#data['lemm_text'] = pd.Series(sentences).apply(lambda x:" ".join(x))

In [5]:
#Файл с лемматизированными словами выгрузил, чтобы не возвращаться к данной операции
#data.to_csv('data_with_lemma.csv', index=False)

In [6]:
#Загрузка файла с лемматизированным текстом по пункту выше
try:
    data = pd.read_csv('data_with_lemma.csv')
except:
    data = pd.read_csv(r'C:\Users\Вадим и Света\Downloads\data_with_lemma.csv')

data

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...


In [7]:
#стопслова
stopwords = set(nltk_stopwords.words('english'))

## Обучение

In [8]:
features = data['lemm_text']
target = data['toxic']

In [9]:
#Разделение на трейн и тест, валидационной выборки делать не буду, применим кросс-валидацию
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=SEED) 

In [10]:
# перевод выборок текста в Tf-idf векторную матрицу
count_tf_idf = TfidfVectorizer(stop_words=stopwords, sublinear_tf=True)
X_train = count_tf_idf.fit_transform(X_train) #Обучение и трансформация на трейне
X_test = count_tf_idf.transform(X_test) # Трансформация тестовой части на обученной в предыдущем пункте модели векторизации
print(X_train.shape, X_test.shape)

(119678, 145527) (39893, 145527)


### LogisticRegression

In [11]:
# Создаем Pipeline с предварительным выбором по матрице наиболее влияющих на предсказания признаков
pipeline = make_pipeline(SelectKBest(chi2, k=25000),
                         LogisticRegression(C=1.05, random_state=SEED, class_weight='balanced', solver='saga', max_iter=400))
result = cross_val_score(pipeline,
                        X_train, y_train,
                        scoring='f1',
                        cv=3, n_jobs=-1,
                       ).mean()
print(f'Метрика F1 на моедли LogisticRegression {result}')

Метрика F1 на моедли LogisticRegression 0.7440708330865068


In [12]:
pipeline_svc = make_pipeline(SelectKBest(chi2, k=30000),
                         LinearSVC(C=0.8, random_state=SEED, class_weight={0:0.2,1:0.8}, max_iter=400))
result = cross_val_score(pipeline_svc,
                        X_train, y_train,
                        scoring='f1',
                        cv=3, n_jobs=-1,
                       ).mean()
print(f'Метрика F1 на моедли LinearSVC {result}')

Метрика F1 на моедли LinearSVC 0.7778726350844355


Из всех рссматриваемых моделей LinearSVC наиболее точная

In [21]:
def objective(trial, data=X_train, target=y_train):
    params = {
        'max_depth': trial.suggest_int('max_depth',-1, 25),
        'num_leaves': trial.suggest_int('num_leaves', 30, 129, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', 
                                                   [0.005,0.006,0.008,0.01,0.015,0.02,0.03]),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    }

    pipeline = make_pipeline(SelectKBest(chi2, k=2000),
                       lgb.LGBMClassifier(**params, random_state = SEED, n_jobs=-1,class_weight='balanced', n_estimators=20))
    score = cross_val_score(pipeline,
                             data, target,
                             scoring='f1',
                             cv=3, n_jobs=-1
                            ).mean()
    return score


In [22]:

study_1 = optuna.create_study(direction='maximize')  
study_1.optimize(objective, n_trials=50)              # Количество итераций = 5

optuna_params = study_1.best_params 

pipeline = make_pipeline(SelectKBest(chi2, k=25000),
                       lgb.LGBMClassifier(**optuna_params, random_state = SEED, n_jobs=-1,class_weight='balanced', n_estimators=2000))
result = cross_val_score(pipeline,
                        X_train, y_train,
                        scoring='f1',
                        cv=3, n_jobs=-1,
                       ).mean()
print(f'Метрика F1 на моедли LGBMClassifier {result}')

[I 2022-05-28 00:47:09,790] A new study created in memory with name: no-name-cc899319-3299-470b-869e-0e0c101a8689
[I 2022-05-28 00:47:14,037] Trial 0 finished with value: 0.643888079632378 and parameters: {'max_depth': 24, 'num_leaves': 60, 'learning_rate': 0.02, 'reg_alpha': 0.7991943785534944, 'reg_lambda': 0.013588009832317609}. Best is trial 0 with value: 0.643888079632378.
[I 2022-05-28 00:47:16,179] Trial 1 finished with value: 0.3611906178771045 and parameters: {'max_depth': 1, 'num_leaves': 96, 'learning_rate': 0.015, 'reg_alpha': 0.005143167527303353, 'reg_lambda': 0.6021200913829953}. Best is trial 0 with value: 0.643888079632378.
[I 2022-05-28 00:47:18,693] Trial 2 finished with value: 0.548684667265895 and parameters: {'max_depth': 6, 'num_leaves': 81, 'learning_rate': 0.02, 'reg_alpha': 5.5958319178695515, 'reg_lambda': 0.010660605282743853}. Best is trial 0 with value: 0.643888079632378.
[I 2022-05-28 00:47:21,606] Trial 3 finished with value: 0.5887707081172641 and param

Метрика F1 на моедли LGBMClassifier 0.7402280174202255


### Тестирование лучшей модели

In [22]:
result = cross_val_score(pipeline_svc,
                        X_test, y_test,
                        scoring='f1',
                        cv=3, n_jobs=-1,
                       ).mean()
print(f'Метрика F1 на моедли LinearSVC на тестовой выборке {result}')

Метрика F1 на моедли LinearSVC на тестовой выборке 0.7563501895187296


## Выводы

Из выше рассмотренных моделей достичь показателя F1 на тренинговой и тестовой выборке удалось на моделе LinearSVC.

In [21]:
pipeline_svc.fit(X_train, y_train)
predict = pipeline_svc.predict(X_test)
c_matrix = confusion_matrix(y_test,predict)
print(f''' Верно положительных комментариев предсказаний: {c_matrix[0,0]}
Ошибочно положительных комментариев предсказаний: {c_matrix[1,0]}
Ошибочно негативных комментариев предсказаний: {c_matrix[0,1]}
Верно негативных комментариев предсказаний: {c_matrix[1,1]}''')


 Верно положительных комментариев предсказаний: 34920
Ошибочно положительных комментариев предсказаний: 839
Ошибочно негативных комментариев предсказаний: 838
Верно негативных комментариев предсказаний: 3296


Количество ложно-положительных и ложно-отрицательных предсказаний практически одинаково, модель верно определяет примерно 80% негативных комментраиев, что говорит что модель работоспособна и сбалансированна. Модель LinearSVC оптимальна для текущей задачи.